# Load and save stock data with database

In [1]:
import os
import sys

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from tqdm import tqdm

from src.stock_data_manager import StockDataManager
from src.utils import Utils

In [2]:
sdm1 = StockDataManager(
    db_user=os.environ["MYSQL_USER"],
    db_password=os.environ["MYSQL_PASSWORD"],
    db_host="localhost",
    db_port="3306",
    db_name="STOCK_RAW",
    alpaca_api_key=os.environ["ALPACA_API_KEY"],
    alpaca_secret_key=os.environ["ALPACA_SECRET_KEY"],
)

sdm2 = StockDataManager(
    db_user=os.environ["MYSQL_USER"],
    db_password=os.environ["MYSQL_PASSWORD"],
    db_host="localhost",
    db_port="3306",
    db_name="STOCK_PROCESSED",
    alpaca_api_key=os.environ["ALPACA_API_KEY"],
    alpaca_secret_key=os.environ["ALPACA_SECRET_KEY"],
)

In [3]:
start_date = "2020-01-01"
end_date = "2024-10-01"
SP500_list_path = "data/tickers.json"

In [4]:
tickers = Utils.load_SP500_list(path=SP500_list_path)

In [5]:
for ticker in tqdm(tickers):

    data = sdm1.get_data_from_alpaca(
        ticker=ticker, start_date=start_date, end_date=end_date
    )
    sdm1.save_to_db(data)

    data = Utils.format_data(stock_data=data)
    data = Utils.adjust_for_stock_splits(
        stock_data=data,
        split_dates=Utils.get_split_dates(
            ticker=ticker, start_date=start_date, end_date=end_date
        ),
    )
    sdm2.save_to_db(data)

100%|██████████| 503/503 [13:54<00:00,  1.66s/it]
